In [ ]:
# Required Libraries
import pandas as pd
import pyodbc
import requests

# Configuration Variables
API_KEY = '1afe5cda384f0d71887086d3367612ce'
SQL_Server = 'MANISH-LAPTOP'
DATABASE = 'CB-Database'
USERNAME = ''
PASSWORD = ''

# Function to Extract Data from TMDb API
def extract_data(api_key):
    url = f'https://api.themoviedb.org/3/movie/top_rated?api_key={api_key}&language=en-US&page=1'
    response = requests.get(url)  # Corrected to `requests` instead of `request`
    
    if response.status_code == 200:
        return response.json()['results'][:100]  # Changed 'result' to 'results' to match the API response structure
    else:
        print('Error Fetching data from TMDb API:', response.status_code)
        return []

# Function to Transform Data into DataFrame
def transform_data(raw_data):
    df = pd.DataFrame(raw_data)
    df = df[['id', 'title', 'overview', 'release_date', 'popularity', 'vote_average', 'vote_count']]
    df.columns = ['Movie_id', 'Title', 'Overview', 'Release_date', 'Popularity', 'Vote_average', 'Vote_count']
    return df

# Function to Load Data into SQL Server
def load_data_to_sql(df):
    connection_string = f'DRIVER={{SQL Server}};SERVER={SQL_Server};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'
    
    # Establish SQL Server connection
    with pyodbc.connect(connection_string) as conn:
        cursor = conn.cursor()
        
        # Create Table if it doesn't exist
        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Top_Movies' AND xtype='U')
        CREATE TABLE Top_Movies (
            MOVIE_ID INT PRIMARY KEY,
            MOVIE_TITLE NVARCHAR(255) NOT NULL,
            OVERVIEW TEXT,
            RELEASE_DATE DATE,
            POPULARITY FLOAT,
            VOTE_AVERAGE FLOAT,
            VOTE_COUNT INT
        )
        ''')
        conn.commit()  # Commit the table creation
        
        # Insert Data into the Table
        for index, row in df.iterrows():
            cursor.execute('''
            INSERT INTO Top_Rated_Movies (MOVIE_ID, MOVIE_TITLE, OVERVIEW, RELEASE_DATE, POPULARITY, VOTE_AVERAGE, VOTE_COUNT)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', row['Movie_id'], row['Title'], row['Overview'], row['Release_date'], row['Popularity'], row['Vote_average'], row['Vote_count'])
        
        conn.commit()  # Commit all insertions

# Main ETL function to run the whole process
def run_etl():
    raw_data = extract_data(API_KEY)
    if raw_data:  # Check if data was successfully extracted
        transformed_data = transform_data(raw_data)
        load_data_to_sql(transformed_data)
        print('ETL process completed successfully.')
    else:
        print('No data to process.')

# Run the ETL process
run_etl()
